# 現代制御
## Topics
* システムの状態方程式表現
* 状態遷移行列
* 安定性
* 可制御性
* 可観測性
* フィードバック
* 最適レギュレータ
* サーボ系
* オブザーバ

## システムの状態方程式表現
状態，入力，出力ベクトルはそれぞれ, $x(t) \in R^n, u(t) \in R^m, y(t) \in R^r$ とし，
定数行列$A \in R^{n\times n}, B \in R^{n\times m}, C \in R^{r\times n}$によって，次のような線形システムが表現される．

$$
\begin{array}
%%\dot{x} &= Ax(t) + Bu(t) \\
\frac{dx}{dt} &= Ax(t) + Bu(t) \\
y &= Cx(t)
\end{array}
$$


In [4]:
import numpy as np
class LinearSystem:
    def __init__(self, A, B, C):
        self.A = A
        self.B = B
        self.C = C
        
    def dot(self, x, u):
        return self.A*x + self.B*u
    
    def output(self, x):
        return self.C*x
    

        

## 状態遷移行列
行列指数関数$e^{At}$を状態遷移行列と呼ぶ．その定義は，
$$
e^{At} = I + At + \frac{1}{2!}A^2t^2 + \cdots + \frac{1}{k!}A^kt^k + \cdots
$$
となる．状態遷移行列を得る主な方法はラプラス逆変換によるものである．

## 安定性
正方行列$A \in R^{n\times n}$の固有値$\lambda \in C$, 固有ベクトル$v \in R^n$は以下の式を満たす．
$$
Av_i = \lambda_i, i = 1, 2, \cdots, n
$$

固有値$\lambda_i$の実部が全て負であるとき，そのシステムは安定である．

## 可制御性
システムが可制御であることの必要十分条件は，可制御性行列$U_c \in R^{n\times nm}$のランクがシステムの次元nと等しくなることである．
$$
U_c = [B\quad AB\quad A^2B\quad \cdots\quad A^{n-1}B]
$$

## 可観測性 
システムが可観測であることの必要十分条件は，可観測性行列$U_o \in R^{rn\times n}$のランクがシステムの次元nと等しくなることである．
$$
U_o = \left[
\begin{array}
CA\\
CA^2\\
\vdots\\
CA^{n-1}
\end{array}
\right]
$$

## フィードバック 


## 最適レギュレータ


## サーボ系 


## オブザーバ 
